# EDA WORKFLOW

## Read the Data Dictionary!
make sure you know what is the target value
seperate numerical and categorical values

## Write and check assumptions

## Look at rows
df.head() / df.tail() / df.sample() 

## Look at missing values
df.isnull().sum()

## General statistics:
df.info()
df.describe(include='all')


## plot distribpution
(box plot /  histogram)

## plot inner correlations
(heatmap)

## show corraltion to the target value

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns